In [1]:
import pickle

In [3]:
# в файле - список комментариев, давайте прочитаем его

with open('comments.list', 'rb') as file:
    comments = pickle.load(file)

In [4]:
# в NLTK уже имеются списки стоп-слов для разных языков
# давайте им воспользуемся
from nltk.corpus import stopwords

# токенизацию будем выполнять при помощи функции word_tokenize
from nltk.tokenize import word_tokenize

# будем осуществлять стеммизацию слов при помощи RussianStemmer
from nltk.stem.snowball import RussianStemmer

In [7]:
stop_words = set(stopwords.words('russian'))

In [8]:
stemmer = RussianStemmer()

In [9]:
def preprocess(comment):
    # сначала комментарий преобразуем к нижнему регистру
    # заглавные буквы нам ни к чему
    comment = comment.lower()

    # токенизируем текст, получив список токенов
    tokens = word_tokenize(comment)

    # в списке оставим только те токены, что являются словами (выкинув, например, знаки препинания)
    # также выкинем стоп-слова
    # всё, что осталось, подвергнем стеммизации
    tokens = [stemmer.stem(t) for t in tokens if t.isalpha() and t not in stop_words]

    return tokens

In [12]:
comments = [preprocess(comm) for comm in comments]

In [14]:
import gensim

In [15]:
model = gensim.models.Word2Vec(comments,  # на вход подаём список "предложений"
                               size=50,  # размерность эмбеддинга
                               window=5,  # размер окна в word2vec
                               min_count=5,  # минимальное количество раз, которое должно встретиться слово
                               sg=1,  # в качестве алгоритма выбираем skip-gram, т.к. датасет небольшой
                               iter=50,  # количество эпох обучения
                               workers=4)  # количество потоков (укажите, исходя из количества ядер вашего процессора)

In [16]:
model.save('comments_model.w2v')

In [ ]:
#Задание 1

In [17]:
model.wv.most_similar(positive=stemmer.stem('аналитик'), topn=5)

[('политолог', 0.7192782163619995),
 ('программист', 0.6743959188461304),
 ('программирован', 0.6600313186645508),
 ('фактик', 0.6451061367988586),
 ('эксперт', 0.6367471218109131)]

In [ ]:
#Задание 2

In [18]:
# в аргументе negative указываем слово, которое будем вычитать

model.wv.most_similar(positive=[stemmer.stem('модератор'), stemmer.stem('добро')], negative=[stemmer.stem('зло')], topn=1)

[('supportcommunity', 0.712459146976471)]

In [ ]:
#Задание 3

In [19]:
model.wv.doesnt_match([stemmer.stem('пользователь'), 
                       stemmer.stem('модератор'), 
                       stemmer.stem('админ'), 
                       stemmer.stem('человек'), 
                       stemmer.stem('пикабушник')])

c:\users\julia\appdata\local\programs\python\python38\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'человек'

In [ ]:
#Задание 4

In [21]:
model.wv.most_similar_to_given(stemmer.stem('пикабушник'), [stemmer.stem('добрый'), 
                                                      stemmer.stem('грустный'), 
                                                      stemmer.stem('злой'), 
                                                      stemmer.stem('скандальный'), 
                                                      stemmer.stem('весёлый')])

'добр'